# Log Parsing Doesn't Need to be Difficult

<i>Note: The log data in this guide is generated by a python script I wrote, which you can find in the same folder as this notebook (you should go through the guide before looking at it!).</i>

Logs are a type of structured data. When a process adds an event to its log file, it follows a consistent, if in some cases unnecessarily complex, template. This is great news! You don't need any fancy word embeddings, dependency parses, or part-of-speech taggers to extract information from log data; all you need is knowledge of the log's structure and a bit of scripting knowledge.

In this guide, I'm going to show how to capture the structure of a log using regular expressions and python without prior knowedge about the log format. I'll do my best to explain what's happening in the regex I use, but this is not a regex tutorial and you should learn a bit about regex before reading. A great cheat sheet / general resource is available here: https://www.rexegg.com/regex-quickstart.html Additionally, the second chapter of Jurafsky and Martin's <i>Speech and Language Processing</i> provides a good introduction to writing regex. A draft of the third edition is currently available online for free: https://web.stanford.edu/~jurafsky/slp3/2.pdf

## Loading the Data

We'll start by looking at the data. At this point, I'd usually do a bit of exploratory analysis, but for the sake of the tutorial I'm going to assume that this is the only data I've seen so far.

In [1]:
with open("logd.log") as log:
    log_str = log.read()
    log_list = log_str.split("\n")
    for line in log_list[:10]:
        print(line)

1505374286 [logd] Failed password for account root from 192.168.142.83
1505374294 [logd] Failed password for account guest from 192.168.27.109
1505374298 [logd] Failed password for account adm from 192.168.142.83
1505374304 [logd] Failed password for account root from 192.168.163.95
1505374313 [logd] Failed password for account root from 192.168.90.32
1505374323 [logd] Failed password for account info from 192.168.164.209
1505374324 [logd] Failed password for account root from 192.168.79.44
1505374329 [logd] Failed password for account root from 192.168.28.238
1505374330 [logd] Failed password for account adm from 192.168.163.95
1505374338 [logd] Failed password for account root from 192.168.194.4


This seems pretty simple; I see a timestamp, followed by the process name, followed by "Failed password for account" then a username and ip. That makes for 3 variables: timestamp, username, and ip. When I write my first regex pattern, I'm going to make sure to ONLY account for these 3 variables, using the most restrictive pattern I can think of. You'll see why I do this in a moment.

For this guide in particular, I'm looking specifically at line-by-line data. For other types of data, you may have to modify your expression a bit. For JSON data, you can simply use the `json` module and skip to the information extraction section of this guide.

<i>regex notes: `d{10}` matches ten digits. I write `\[` instead of `[` because `[` is a special character in regex and needs to be escaped in order to be matched properly. `\w+` matches 1 or more "word" characters, but not a space. `^` and `$` mark the beginning and end of a line. `re.M` is a python-specific flag that is needed to match `^` and `$` properly.</i>

In [2]:
import re

ip = "\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}"
pattern = "^\d{10} \[logd\] Failed password for account \w+ from " + ip + "$"
expr = re.compile(pattern, re.M)

matches = [match.group(0) for match in expr.finditer(log_str)]

for match in matches[:10]:
    print(match)

# I subtract 1 from len(log_list) because there's an empty line at the bottom of the file
print("\nTotal events:", len(log_list) - 1)
print("Captured here:", len(matches))

1505374286 [logd] Failed password for account root from 192.168.142.83
1505374294 [logd] Failed password for account guest from 192.168.27.109
1505374298 [logd] Failed password for account adm from 192.168.142.83
1505374304 [logd] Failed password for account root from 192.168.163.95
1505374313 [logd] Failed password for account root from 192.168.90.32
1505374323 [logd] Failed password for account info from 192.168.164.209
1505374324 [logd] Failed password for account root from 192.168.79.44
1505374329 [logd] Failed password for account root from 192.168.28.238
1505374330 [logd] Failed password for account adm from 192.168.163.95
1505374338 [logd] Failed password for account root from 192.168.194.4

Total events: 10000
Captured here: 9548


It looks like the "Failed password" pattern only accounts for about 95% of the data. Our goal is to write a pattern that accounts for 100% of the data, so it's clear just from this that we're missing something. To find out what, I'm going to write a pattern that captures everything this pattern doesn't.

<i>regex notes: `(?!text)` is a negative lookaround. It ensures that anything starting with "text" is not matched. In this case, I put my whole pattern in the lookaround to match everything the pattern doesn't cover. `.*` will match 0 or more non-newline characters.</i>

In [3]:
invert = "(?!" + pattern + ")^.*$"
expr = re.compile(invert, re.M)

matches = [match.group(0) for match in expr.finditer(log_str)]
for match in matches[:10]:
    print(match)

1505374340 [logd] Error: unexpected character '%'
1505374418 [logd] Failed public key for account test from 192.168.163.95
1505374468 [logd] Error: unexpected character '%'
1505374483 [logd] repeated 5 times: [ Failed password for account root from 192.168.107.11 ]
1505374598 [logd] repeated 2 times: [ Failed password for account admin from 192.168.142.83 ]
1505374635 [logd] Failed public key for account root from 192.168.253.72
1505374830 [logd] Error: Connection closed
1505374947 [logd] Error: Connection closed
1505375121 [logd] Failed public key for account mysql from 192.168.164.209
1505375296 [logd] Successful password for account root from 192.168.163.95


This shows some of the log events that don't match my intentionally restrictive format. This is why I made the pattern restrictive in the first place; from this, I can quickly get an idea of what data I was missing before. If my pattern wasn't restrictive enough, I may have accidentally matched a log event I didn't account for and as a result fail to notice an important outlier.

To start out with, I'm going to take care of the "repeated x times" event. This is a case were the best solution (in my opinion) is to modify the log itself rather than accounting for the event in the parser. I'm going to use some regex to simply duplicate each "repeated" event by the number of times its repeated:

<i>regex notes: I'm now using capture groups to extract specific information from the log. By surrounding part of the pattern in parenthesis, I "capture" that part of the pattern and can acces it later. Because the number of repetitions is the second capture group, I multiply the reconstructed line with `int(m.group(2))`</i>

In [4]:
fixed_log = re.sub("^(\d{10} \[logd\] )repeated (\d) times: \[ (.*?) \]\n",
                  lambda m: (m.group(1) + m.group(3) + "\n")*int(m.group(2)),
                  log_str, flags=re.M)
log_list = fixed_log.split("\n")
print(len(log_list) - 1)

10419


You'll notice that the log has grown from 10000 events to 10419. If you're viewing this guide as a jupyter notebook, I encourage you to modify the code block above to print out some data and verify the substitution was correct (I'm too lazy to :)).

Now that we've taken care of the repetitions, let's peak back at the unmatched data:

In [5]:
# I don't need to change the expression, so I won't redefine it
# notice that I'm now using fixed_log instead of log_str

matches = [match.group(0) for match in expr.finditer(fixed_log)]
for match in matches[:10]:
    print(match)

1505374340 [logd] Error: unexpected character '%'
1505374418 [logd] Failed public key for account test from 192.168.163.95
1505374468 [logd] Error: unexpected character '%'
1505374635 [logd] Failed public key for account root from 192.168.253.72
1505374830 [logd] Error: Connection closed
1505374947 [logd] Error: Connection closed
1505375121 [logd] Failed public key for account mysql from 192.168.164.209
1505375296 [logd] Successful password for account root from 192.168.163.95
1505375387 [logd] Failed public key for account test from 192.168.240.202
1505375428 [logd] Error: unexpected character '%'


There's two quick fixes here: not all log events are failures, and not all login methods use a password (some use public key). I'll take care of those first.

In [6]:
pattern = "^\d{10} \[logd\] \w+ .* for account \w+ from " + ip + "$"
expr = re.compile(pattern, re.M)

matches = [match.group(0) for match in expr.finditer(fixed_log)]

for match in matches[:10]:
    print(match)

print("\nTotal events:", len(log_list) - 1)
print("Captured here:", len(matches))

1505374286 [logd] Failed password for account root from 192.168.142.83
1505374294 [logd] Failed password for account guest from 192.168.27.109
1505374298 [logd] Failed password for account adm from 192.168.142.83
1505374304 [logd] Failed password for account root from 192.168.163.95
1505374313 [logd] Failed password for account root from 192.168.90.32
1505374323 [logd] Failed password for account info from 192.168.164.209
1505374324 [logd] Failed password for account root from 192.168.79.44
1505374329 [logd] Failed password for account root from 192.168.28.238
1505374330 [logd] Failed password for account adm from 192.168.163.95
1505374338 [logd] Failed password for account root from 192.168.194.4

Total events: 10419
Captured here: 10272


I could be far more restrictive with how I'm matching the login type, but I want to keep things fairly concise for this guide. In this case, `.*` works fine because `for account` comes immediately after and prevents the `*` quantifier from consuming the whole line.

Let's check up on the invert pattern again:

In [7]:
invert = "(?!" + pattern + ")^.*$"
expr = re.compile(invert, re.M)

matches = [match.group(0) for match in expr.finditer(fixed_log)]
for match in matches[:10]:
    print(match)

1505374340 [logd] Error: unexpected character '%'
1505374468 [logd] Error: unexpected character '%'
1505374830 [logd] Error: Connection closed
1505374947 [logd] Error: Connection closed
1505375428 [logd] Error: unexpected character '%'
1505375525 [logd] Error: Connection closed
1505375791 [logd] Error: Connection closed
1505375981 [logd] Error: Connection closed
1505376008 [logd] Error: Connection closed
1505376681 [logd] Error: Connection closed


It looks like all that's left is error messages. I'm going to modify my expression match the entire error message when there's an error to keep things simple (It's probably the better way to do things with this log anyway)

<i>regex notes: `(?:text)` is a non-capturing group, `|` is an or switch. Don't worry about the capturing part, it just prevents the or switch from applying to the whole line.</i>

In [8]:
pattern = "^\d{10} \[logd\] (?:\w+ .* for account \w+ from " + ip + "|Error: .*)$"
expr = re.compile(pattern, re.M)

matches = [match.group(0) for match in expr.finditer(fixed_log)]

for match in matches[:10]:
    print(match)

print("\nTotal events:", len(log_list) - 1)
print("Captured here:", len(matches))

1505374286 [logd] Failed password for account root from 192.168.142.83
1505374294 [logd] Failed password for account guest from 192.168.27.109
1505374298 [logd] Failed password for account adm from 192.168.142.83
1505374304 [logd] Failed password for account root from 192.168.163.95
1505374313 [logd] Failed password for account root from 192.168.90.32
1505374323 [logd] Failed password for account info from 192.168.164.209
1505374324 [logd] Failed password for account root from 192.168.79.44
1505374329 [logd] Failed password for account root from 192.168.28.238
1505374330 [logd] Failed password for account adm from 192.168.163.95
1505374338 [logd] Failed password for account root from 192.168.194.4

Total events: 10419
Captured here: 10415


Only 4 events left! This is why I like to do things one at a time like this. Those 4 outlier events would be very difficult to find by just looking through the 10000 event log.

In [9]:
invert = "(?!" + pattern + ")^.*$"
expr = re.compile(invert, re.M)

matches = [match.group(0) for match in expr.finditer(fixed_log)]
for match in matches[:10]:
    print(match)

1505380274 [logd] Failed password for account ec2-user from 192.168.101.23
1505394385 [logd] Failed password for account ec2-user from 192.168.142.83
1505406169 [logd] Failed password for account ec2-user from 192.168.190.54
1505421194 [logd] Failed password for account ec2-user from 192.168.142.83



Unfortunately, this isn't a very interesting outlier (don't worry, we'll get to those!). It's just a username which isn't matched by `\w+`. It's an easy fix, I'll just make the username portion also match dashes.

<i>regex note: `[abc]` will match any character in the brackers (in this case, a, b, or c)</i>

In [10]:
pattern = "^\d{10} \[logd\] (?:\w+ .* for account [\w-]+ from " + ip + "|Error: .*)$"
expr = re.compile(pattern, re.M)

matches = [match.group(0) for match in expr.finditer(fixed_log)]

for match in matches[:10]:
    print(match)

print("\nTotal events:", len(log_list) - 1)
print("Captured here:", len(matches))

1505374286 [logd] Failed password for account root from 192.168.142.83
1505374294 [logd] Failed password for account guest from 192.168.27.109
1505374298 [logd] Failed password for account adm from 192.168.142.83
1505374304 [logd] Failed password for account root from 192.168.163.95
1505374313 [logd] Failed password for account root from 192.168.90.32
1505374323 [logd] Failed password for account info from 192.168.164.209
1505374324 [logd] Failed password for account root from 192.168.79.44
1505374329 [logd] Failed password for account root from 192.168.28.238
1505374330 [logd] Failed password for account adm from 192.168.163.95
1505374338 [logd] Failed password for account root from 192.168.194.4

Total events: 10419
Captured here: 10419


Okay, so we matched everything... now what?

## Information Extraction

To begin with, I'm going to surround every variable of interest using capture groups. This will make everything nicely structured and easy to access.

<i>regex note: I'm now using match.groups() instead of match.group(0) so I get the capture groups instead of the full match</i>

In [11]:
pattern = "^(\d{10}) \[logd\] (?:(\w+) (.*) for account ([\w-]+) from (" + ip + ")|Error: (.*))$"
expr = re.compile(pattern, re.M)

matches = [match.groups() for match in expr.finditer(fixed_log)]


for match in matches[:10]:
    print(match)

('1505374286', 'Failed', 'password', 'root', '192.168.142.83', None)
('1505374294', 'Failed', 'password', 'guest', '192.168.27.109', None)
('1505374298', 'Failed', 'password', 'adm', '192.168.142.83', None)
('1505374304', 'Failed', 'password', 'root', '192.168.163.95', None)
('1505374313', 'Failed', 'password', 'root', '192.168.90.32', None)
('1505374323', 'Failed', 'password', 'info', '192.168.164.209', None)
('1505374324', 'Failed', 'password', 'root', '192.168.79.44', None)
('1505374329', 'Failed', 'password', 'root', '192.168.28.238', None)
('1505374330', 'Failed', 'password', 'adm', '192.168.163.95', None)
('1505374338', 'Failed', 'password', 'root', '192.168.194.4', None)


Our new `matches` list gives us everything we need to know about the data. For example, if we want to look at all the error messages, we just have to look at the sixth capture group wherever the second capture group is empty (didn't match success/failure group):

In [12]:
print(set(m[5] for m in matches if m[1] == None))

{'Connection closed', "unexpected character '%'"}


That isn't very intuitive, so I'm going to name my capture groups for easy access:

<i>regex note: `(?P<name>match)` is the syntax for naming a capture group. Note that I'm now using `groupdict()` because the matches provide a dictionary</i>

In [13]:
pattern = "^(?P<time>\d{10}) \[logd\] (?:(?P<result>\w+) (?P<method>.*) for account (?P<user>[\w-]+) from (?P<ip>" + ip + ")|Error: (?P<error>.*))$"
expr = re.compile(pattern, re.M)

matches = [match.groupdict() for match in expr.finditer(fixed_log)]

for match in matches[:10]:
    print(match)

{'time': '1505374286', 'result': 'Failed', 'method': 'password', 'user': 'root', 'ip': '192.168.142.83', 'error': None}
{'time': '1505374294', 'result': 'Failed', 'method': 'password', 'user': 'guest', 'ip': '192.168.27.109', 'error': None}
{'time': '1505374298', 'result': 'Failed', 'method': 'password', 'user': 'adm', 'ip': '192.168.142.83', 'error': None}
{'time': '1505374304', 'result': 'Failed', 'method': 'password', 'user': 'root', 'ip': '192.168.163.95', 'error': None}
{'time': '1505374313', 'result': 'Failed', 'method': 'password', 'user': 'root', 'ip': '192.168.90.32', 'error': None}
{'time': '1505374323', 'result': 'Failed', 'method': 'password', 'user': 'info', 'ip': '192.168.164.209', 'error': None}
{'time': '1505374324', 'result': 'Failed', 'method': 'password', 'user': 'root', 'ip': '192.168.79.44', 'error': None}
{'time': '1505374329', 'result': 'Failed', 'method': 'password', 'user': 'root', 'ip': '192.168.28.238', 'error': None}
{'time': '1505374330', 'result': 'Failed'

This structure makes it very easy to find summary statistics. For example, how many failed login attempts for the root user tried using a public key?

In [14]:
print(len([m for m in matches if m["result"] == "Failed" and m["method"] == "public key" and m["user"] == "root"]))

33


Let's say instead we want to know how many unique ips failed an ssh key login for root. No problem! We just need a slight adjustment to the code.

In [15]:
print(len(set([m["ip"] for m in matches
               if m["result"] == "Failed" and m["method"] == "public key" and m["user"] == "root"])))

13


Looks like several ips tried logging in with a public key multiple times, so let's see which ip was the most common of the 13 that tried. Figuring out which ip made the most attempts will be a bit more work, but not much.

In [16]:
fail_ssh_ips = [m["ip"] for m in matches
                if m["result"] == "Failed" and m["method"] == "public key" and m["user"] == "root"]
ip_dict = {}
for ip in fail_ssh_ips:
    if ip not in ip_dict:
        ip_dict[ip] = 1
    else:
        ip_dict[ip] += 1

print(ip_dict)
print("Most common ip:", max(ip_dict, key=lambda x: ip_dict[x]))

{'192.168.253.72': 1, '192.168.90.67': 1, '192.168.142.83': 6, '192.168.101.23': 7, '192.168.240.202': 3, '192.168.116.192': 3, '192.168.53.38': 1, '192.168.107.11': 3, '192.168.163.95': 3, '192.168.164.209': 2, '192.168.157.155': 1, '192.168.171.229': 1, '192.168.28.238': 1}
Most common ip: 192.168.101.23


Next, lets see what the most common username was for successful logins (of any type)

In [17]:
success_unames = [m["user"] for m in matches if m["result"] == "Successful"]
uname_dict = {}
for uname in success_unames:
    if uname not in uname_dict:
        uname_dict[uname] = 1
    else:
        uname_dict[uname] += 1

print(uname_dict)
print("Most common username:", max(uname_dict, key=lambda x: uname_dict[x]))

{'root': 25, 'guest': 5, 'test': 6, 'info': 3, 'admin': 8, 'adm': 1, 'mysql': 1}
Most common username: root


Finally, let's extract unique ips from a certain timeframe (selected arbitrary)

In [18]:
time_ips = [m["ip"] for m in matches if int(m["time"]) > 1505390000 and int(m["time"]) < 1505400000]
print("Unique ips in this timespan:", len(set(time_ips)))
print("Number of events in this timespan:", len(time_ips))
print("Total unique ips:", len(set([m["ip"] for m in matches])))

Unique ips in this timespan: 35
Number of events in this timespan: 2093
Total unique ips: 41


I hope that it's clear at this point that, using this data structure, we can extract almost any statistic of interest from the log. Even better, everything is consistent. If we do make an error at some point (which is bound to occur), fixing the parser for one query will fix it for all the queries. This makes it much easier to identify what went wrong if a statistic is calculated incorrectly.

## Finding Outliers

Finally, the most interesting part of the process! There are three outliers I hid in this log. When looking for outliers, I like to print out all my variables one at a time to get an idea of their range. Let's start with usernames:

In [19]:
print(set([m["user"] for m in matches]))

{'pi', 'ftp', 'user', None, 'administrator', 'adm', 'guest', 'oracle', 'ec2-user', 'admin', 'test', 'root', 'puppet', 'ansible', 'info', 'azureuser', 'mysql'}


Nothing too interesting in there. What about login methods?

In [20]:
print(set([m["method"] for m in matches]))

{None, 'password', 'session id', 'public key'}


`None` comes from events which are errors. You'll notice, however, that something in there is unfamiliar. We didn't see `session id` when looking through the data, how common could it be?

In [21]:
print([m for m in matches if m["method"] == "session id"])

[{'time': '1505388095', 'result': 'Failed', 'method': 'session id', 'user': 'root', 'ip': '192.168.28.238', 'error': None}]


Well, that would explain it. There was only one attempt which tried using a session id! Outliers such as this are easy to find by simply printing out the set of all variables. Next, I'm going to show a far more difficult outlier. There's one outlier I'm skipping; if you're viewing this guide in jupyter, I encourage you to look at range of a few other variables and see if you can find it.

Unfortunately, for continuous data just looking at all the possible values isn't an option. This particular log doesn't have much numerical data, but there's one place where an outlier can show up: the timestamp. To look for outliers in time, I'm first going to get an idea of the range of the data

In [22]:
times = [int(m["time"]) for m in matches]
print("Range:", min(times), "-", max(times))

Range: 1505374286 - 1505423698


I'm not very interested in the absolute time, so I'm just going to subtract the min from everything.

In [23]:
tmin = min(times)
times = [int(m["time"]) - tmin for m in matches]
print("Range:", min(times), "-", max(times))

Range: 0 - 49412


Just to make sure 49412 isn't an outlier, I'm going to use numpy to look at some percentiles (Obviously, because timestamps are sorted, you could just check the second-to-last timestamp. I'm showing this because for other continuous data it's the most basic check):

In [24]:
import numpy as np

p_025 = np.percentile(times, 2.5)
p_50 = np.percentile(times, 50)
p_975 = np.percentile(times, 97.5)

print("95% of the data lies between", int(p_025), "and", int(p_975))
print("The median is", p_50)

95% of the data lies between 1198 and 48117
The median is 24682.0


Based on that output, it looks like the timestamps increase fairly linearly. Let's look at the change in time between each event to check for anything strange there.

In [25]:
delta_t = [times[i] - times[i - 1] for i in range(1, len(times))]

p_025 = np.percentile(delta_t, 2.5)
p_50 = np.percentile(delta_t, 50)
p_975 = np.percentile(delta_t, 97.5)

print("max:", max(delta_t), "min:", min(delta_t))
print("95% of the data lies between", int(p_025), "and", int(p_975))
print("The median is", p_50)

max: 209 min: 0
95% of the data lies between 0 and 10
The median is 5.0


The max here immediately draws my attention. 95% of our data is between 0 and 10, but one of the delays is 209. Let's look at the first few values of the sorted time changes:

In [26]:
print(sorted(delta_t)[-10:])

[10, 10, 10, 10, 10, 10, 10, 10, 10, 209]


Sure enough, there's a very clear outlier here. Our log had a gap of 209 seconds where nothing happened, with the next closest gap being 10 seconds. Strange!

## Wrap-up
To conclude, I would like to re-iterate the steps of the process:
1. Observe some of the data.
2. Write a reasonably restrictive regular expression to capture just that data.
3. Observe the unmatched data and return to (2) until all data is matched
5. Surround all variables of interest with capture groups

This procedure provides a straightforward, comprehensive approach to extracting information from simple log files. It may seem lengthy and convoluted because I spent so much time explaining everything, but in practice this is a fairly quick and brainless process which gives highly accurate results.